In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

sns.set_theme()

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# no text wrapping
display(HTML("<style>.dataframe td { white-space: nowrap; }</style>"))

# pandas formatting
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', 200)

In [2]:
df_hist = pd.read_csv(r'.\csv\trapnet_biologicaldetailing_202302231329.csv', low_memory=False)
df_spec = pd.read_csv(r'.\csv\trapnet_specimen_202302231329.csv', low_memory=False)

# historical data without trivial fields (fields with only one entry)
unmatchable_hist = ['created_at', 'updated_at']
df_hist = df_hist.drop(unmatchable_hist, axis=1).loc[:, df_hist.nunique() > 1]
hist_sample_id_list = sorted(list(df_hist.sample_id.unique()))

# all species in historical data are 79 (atlantic salmon)
unmatchable_spec = ['created_at', 'updated_at', 'created_by_id', 'updated_by_id']
df_spec = df_spec[(df_spec.species_id==79) & (df_spec.sample_id.isin(hist_sample_id_list))].reset_index(drop=True)
df_spec = df_spec.drop(unmatchable_spec, axis=1).loc[:, df_spec.nunique() > 1]

In [3]:
# cast to int where appropriate
df_hist[['fork_length', 'sex_id']] = df_hist[['fork_length', 'sex_id']].astype('Int64')
df_spec[['fork_length', 'sex_id']] = df_spec[['fork_length', 'sex_id']].astype('Int64')

In [4]:
# archived fish data
df_archive = pd.read_csv(r'.\csv\fish_data.csv', low_memory=False)
df_archive['DATETIME'] = pd.to_datetime(df_archive['SITE_EVENT_DATE'].str.split(' ', expand=True)[0], format='%d/%m/%Y')
df_archive['old_id'] = 'GD_' + df_archive['GD_ID'].astype(str)
df_archive = df_archive[['old_id', 'DATETIME'] + [x for x in df_archive.columns if x not in ['SITE_EVENT_DATE', 'DATETIME', 'old_id']]]

# differences between archive and biologicaldetailing
#### conclusions: old_id <-> GD_ID correctly and adequately match samples between the database and the archive

In [8]:
df_merged = pd.merge(df_hist, df_archive, on='old_id')
first_columns = ['sample_id', 'old_id', 'DATETIME']
df_merged = df_merged[first_columns + [x for x in sorted(list(df_merged.columns), key=str.lower) if x not in first_columns]]

In [51]:
# need to join with sample to get dates for comparison
# SAME NUMBER OF ROWS!!!
df_merged.shape, pd.read_csv(r'.\csv\trapnet_biologicaldetailing_202302231329.csv', low_memory=False).shape

((27524, 43), (27524, 23))

In [28]:
df_merged.SURVEY_TYPE.value_counts()

1    27351
3      173
Name: SURVEY_TYPE, dtype: int64

In [29]:
# all atlantic salmon
df_merged.SPECIES_ITIS_CODE.value_counts()

161996    27524
Name: SPECIES_ITIS_CODE, dtype: int64

In [49]:
# about 10x more salmon in the archive file
df_archive[df_archive.SPECIES_ITIS_CODE=='161996'].SPECIES_ITIS_CODE.value_counts()

161996    256154
Name: SPECIES_ITIS_CODE, dtype: int64

In [90]:
comparisons = {
    'age_type': 'AGE_TYPE',
    'fork_length': 'FORK_LENGTH',
    'life_stage_id': 'MATURITY', 
    'sex_id': 'SEX',
    'total_length': 'TOTAL_LENGTH',
    'weight': 'WEIGHT',
    'river_age': 'RIVER_AGE'
}

In [91]:
i = 6
k = list(comparisons.keys())[i] 
v = comparisons[k]
df_merged[first_columns + [k, v]]  # how to check if these are always matched? do it individually

,sample_id,old_id,DATETIME,river_age,RIVER_AGE
0,4390,GD_1,1968-08-09,0.000,0.000
1,4390,GD_2,1968-08-09,0.000,0.000
2,4390,GD_3,1968-08-09,0.000,0.000
3,4390,GD_4,1968-08-09,0.000,0.000
4,4390,GD_5,1968-08-09,0.000,0.000
...,...,...,...,...,...
27519,8001,GD_207005,2000-07-12,0.000,0.000
27520,8001,GD_207006,2000-07-12,0.000,0.000
27521,8001,GD_207011,2000-07-12,1.000,1.000
27522,8001,GD_207012,2000-07-12,1.000,1.000


In [65]:
# Age Type is always the same
df_merged[df_merged.AGE_TYPE.fillna(0) != df_merged.age_type.map({1:'SCALE', 2:'LGTHFREQ'}).fillna(0)]

,sample_id,old_id,DATETIME,age_type,AGE_TYPE,BIOLOGICAL_REMARKS,BIOLOGICAL_SAMPLE,CATCH_FREQUENCY,CATCHMENT_INDEX,CATCHMENT_NAME,FILE_TYPE,FISH_SIZE,FISH_STATUS,fork_length,FORK_LENGTH,FORK_LENGTH_INTERVAL_WIDTH,GD_ID,id,life_stage_id,MATURITY,notes,ORIGIN,RECORD_IDENTIFIER,river_age,RIVER_AGE,SCALE_SAMPLE,SCALE_SAMPLE_ID,SEX,sex_id,SITE,SITE_EVENT_CODE,SPECIES_ITIS_CODE,SPECIES_LIFE_STAGE,status_id,SURVEY,SURVEY_TYPE,SWEEP_NUMBER,total_length,TOTAL_LENGTH,weight,WEIGHT,WEIGHT_PRECISION,WEIGHT_RESOLUTION


In [69]:
# fork length is always the same
df_merged[df_merged.fork_length.fillna(0) != df_merged.FORK_LENGTH.fillna(0).astype(int)]

,sample_id,old_id,DATETIME,age_type,AGE_TYPE,BIOLOGICAL_REMARKS,BIOLOGICAL_SAMPLE,CATCH_FREQUENCY,CATCHMENT_INDEX,CATCHMENT_NAME,FILE_TYPE,FISH_SIZE,FISH_STATUS,fork_length,FORK_LENGTH,FORK_LENGTH_INTERVAL_WIDTH,GD_ID,id,life_stage_id,MATURITY,notes,ORIGIN,RECORD_IDENTIFIER,river_age,RIVER_AGE,SCALE_SAMPLE,SCALE_SAMPLE_ID,SEX,sex_id,SITE,SITE_EVENT_CODE,SPECIES_ITIS_CODE,SPECIES_LIFE_STAGE,status_id,SURVEY,SURVEY_TYPE,SWEEP_NUMBER,total_length,TOTAL_LENGTH,weight,WEIGHT,WEIGHT_PRECISION,WEIGHT_RESOLUTION


In [72]:
# nope.
df_merged.MATURITY.unique()

array([nan])

In [74]:
# Sex is always the same
df_merged[df_merged.SEX.fillna('U') != df_merged.sex_id.fillna(0).map({0: 'U', 1:'M', 2:'F'})]

,sample_id,old_id,DATETIME,age_type,AGE_TYPE,BIOLOGICAL_REMARKS,BIOLOGICAL_SAMPLE,CATCH_FREQUENCY,CATCHMENT_INDEX,CATCHMENT_NAME,FILE_TYPE,FISH_SIZE,FISH_STATUS,fork_length,FORK_LENGTH,FORK_LENGTH_INTERVAL_WIDTH,GD_ID,id,life_stage_id,MATURITY,notes,ORIGIN,RECORD_IDENTIFIER,river_age,RIVER_AGE,SCALE_SAMPLE,SCALE_SAMPLE_ID,SEX,sex_id,SITE,SITE_EVENT_CODE,SPECIES_ITIS_CODE,SPECIES_LIFE_STAGE,status_id,SURVEY,SURVEY_TYPE,SWEEP_NUMBER,total_length,TOTAL_LENGTH,weight,WEIGHT,WEIGHT_PRECISION,WEIGHT_RESOLUTION


In [83]:
# total length is always the same
df_merged[df_merged.total_length.fillna(0) != df_merged.TOTAL_LENGTH.fillna(0)]

,sample_id,old_id,DATETIME,age_type,AGE_TYPE,BIOLOGICAL_REMARKS,BIOLOGICAL_SAMPLE,CATCH_FREQUENCY,CATCHMENT_INDEX,CATCHMENT_NAME,FILE_TYPE,FISH_SIZE,FISH_STATUS,fork_length,FORK_LENGTH,FORK_LENGTH_INTERVAL_WIDTH,GD_ID,id,life_stage_id,MATURITY,notes,ORIGIN,RECORD_IDENTIFIER,river_age,RIVER_AGE,SCALE_SAMPLE,SCALE_SAMPLE_ID,SEX,sex_id,SITE,SITE_EVENT_CODE,SPECIES_ITIS_CODE,SPECIES_LIFE_STAGE,status_id,SURVEY,SURVEY_TYPE,SWEEP_NUMBER,total_length,TOTAL_LENGTH,weight,WEIGHT,WEIGHT_PRECISION,WEIGHT_RESOLUTION


In [82]:
# weight is always the same
df_merged[df_merged.weight.fillna(0) != df_merged.WEIGHT.fillna(0)]

,sample_id,old_id,DATETIME,age_type,AGE_TYPE,BIOLOGICAL_REMARKS,BIOLOGICAL_SAMPLE,CATCH_FREQUENCY,CATCHMENT_INDEX,CATCHMENT_NAME,FILE_TYPE,FISH_SIZE,FISH_STATUS,fork_length,FORK_LENGTH,FORK_LENGTH_INTERVAL_WIDTH,GD_ID,id,life_stage_id,MATURITY,notes,ORIGIN,RECORD_IDENTIFIER,river_age,RIVER_AGE,SCALE_SAMPLE,SCALE_SAMPLE_ID,SEX,sex_id,SITE,SITE_EVENT_CODE,SPECIES_ITIS_CODE,SPECIES_LIFE_STAGE,status_id,SURVEY,SURVEY_TYPE,SWEEP_NUMBER,total_length,TOTAL_LENGTH,weight,WEIGHT,WEIGHT_PRECISION,WEIGHT_RESOLUTION


In [92]:
# weight is always the same
df_merged[df_merged.river_age.fillna(0) != df_merged.RIVER_AGE.fillna(0)]

,sample_id,old_id,DATETIME,age_type,AGE_TYPE,BIOLOGICAL_REMARKS,BIOLOGICAL_SAMPLE,CATCH_FREQUENCY,CATCHMENT_INDEX,CATCHMENT_NAME,FILE_TYPE,FISH_SIZE,FISH_STATUS,fork_length,FORK_LENGTH,FORK_LENGTH_INTERVAL_WIDTH,GD_ID,id,life_stage_id,MATURITY,notes,ORIGIN,RECORD_IDENTIFIER,river_age,RIVER_AGE,SCALE_SAMPLE,SCALE_SAMPLE_ID,SEX,sex_id,SITE,SITE_EVENT_CODE,SPECIES_ITIS_CODE,SPECIES_LIFE_STAGE,status_id,SURVEY,SURVEY_TYPE,SWEEP_NUMBER,total_length,TOTAL_LENGTH,weight,WEIGHT,WEIGHT_PRECISION,WEIGHT_RESOLUTION


# are there any matches between GD_ID and old_id from specimen table?

In [94]:
pd.merge(df_archive, df_spec, on='old_id')

,old_id,DATETIME,SURVEY,SITE,CATCHMENT_NAME,CATCHMENT_INDEX,SURVEY_TYPE,SITE_EVENT_CODE,SWEEP_NUMBER,RECORD_IDENTIFIER,SPECIES_ITIS_CODE,SPECIES_LIFE_STAGE,ORIGIN,FISH_STATUS,FORK_LENGTH,FORK_LENGTH_INTERVAL_WIDTH,TOTAL_LENGTH,WEIGHT,WEIGHT_RESOLUTION,WEIGHT_PRECISION,FILE_TYPE,CATCH_FREQUENCY,FISH_SIZE,RIVER_AGE,AGE_TYPE,SEX,MATURITY,BIOLOGICAL_REMARKS,BIOLOGICAL_SAMPLE,SCALE_SAMPLE,SCALE_SAMPLE_ID,GD_ID,id,fork_length,weight,river_age,notes,sample_id,sex_id,status_id,age_type,sweep_id,life_stage_id,smart_river_age,smart_river_age_type
0,GD_202,1970-06-29,1970,7,MIRAMICHI,2,1,1,1.000,8624004,161996,1731.000,W,RS,33.000,5.000,NaN,NaN,NaN,NaN,1.000,6,FRY,0.000,LGTHFREQ,NaN,NaN,NaN,1.000,NaN,NaN,202,2219941,33,NaN,0.000,None,4404,<NA>,10.000,2.000,305.000,11.000,0.000,2.000
1,GD_202,1970-06-29,1970,7,MIRAMICHI,2,1,1,1.000,8624004,161996,1731.000,W,RS,33.000,5.000,NaN,NaN,NaN,NaN,1.000,6,FRY,0.000,LGTHFREQ,NaN,NaN,NaN,1.000,NaN,NaN,202,2219942,33,NaN,0.000,None,4404,<NA>,10.000,2.000,305.000,11.000,0.000,2.000
2,GD_202,1970-06-29,1970,7,MIRAMICHI,2,1,1,1.000,8624004,161996,1731.000,W,RS,33.000,5.000,NaN,NaN,NaN,NaN,1.000,6,FRY,0.000,LGTHFREQ,NaN,NaN,NaN,1.000,NaN,NaN,202,2219943,33,NaN,0.000,None,4404,<NA>,10.000,2.000,305.000,11.000,0.000,2.000
3,GD_202,1970-06-29,1970,7,MIRAMICHI,2,1,1,1.000,8624004,161996,1731.000,W,RS,33.000,5.000,NaN,NaN,NaN,NaN,1.000,6,FRY,0.000,LGTHFREQ,NaN,NaN,NaN,1.000,NaN,NaN,202,2219944,33,NaN,0.000,None,4404,<NA>,10.000,2.000,305.000,11.000,0.000,2.000
4,GD_202,1970-06-29,1970,7,MIRAMICHI,2,1,1,1.000,8624004,161996,1731.000,W,RS,33.000,5.000,NaN,NaN,NaN,NaN,1.000,6,FRY,0.000,LGTHFREQ,NaN,NaN,NaN,1.000,NaN,NaN,202,2219945,33,NaN,0.000,None,4404,<NA>,10.000,2.000,305.000,11.000,0.000,2.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74661,GD_207010,2000-07-12,2000,53,RESTIGOUCHE,1,1,1,1.000,8563339,161996,1731.000,W,RS,78.000,5.000,NaN,NaN,NaN,NaN,1.000,1,PARR,1.000,LGTHFREQ,NaN,NaN,NaN,1.000,NaN,NaN,207010,2430208,78,NaN,1.000,None,8001,<NA>,10.000,2.000,10861.000,1.000,1.000,2.000
74662,GD_207013,2000-07-12,2000,53,RESTIGOUCHE,1,1,1,1.000,8563340,161996,1731.000,W,RS,88.000,5.000,NaN,NaN,NaN,NaN,1.000,1,PARR,1.000,LGTHFREQ,NaN,NaN,NaN,1.000,NaN,NaN,207013,2430209,88,NaN,1.000,None,8001,<NA>,10.000,2.000,10861.000,1.000,1.000,2.000
74663,GD_207015,2000-07-12,2000,53,RESTIGOUCHE,1,1,1,1.000,8563341,161996,1731.000,W,RS,93.000,5.000,NaN,NaN,NaN,NaN,1.000,1,PARR,1.000,LGTHFREQ,NaN,NaN,NaN,1.000,NaN,NaN,207015,2430210,93,NaN,1.000,None,8001,<NA>,10.000,2.000,10861.000,1.000,1.000,2.000
74664,GD_207016,2000-07-12,2000,53,RESTIGOUCHE,1,1,1,0.500,8563308,161996,1731.000,W,RS,95.000,1.000,NaN,NaN,NaN,NaN,2.000,1,PARR,1.000,LGTHFREQ,NaN,NaN,NaN,1.000,NaN,NaN,207016,2430211,95,NaN,1.000,None,8001,<NA>,10.000,2.000,10860.000,1.000,1.000,2.000


In [95]:
# these need to be excluded I guess... hmmmmm.....

# could we get better matches using historical data instead of dm_apps data?

# samples with potential matches

In [4]:
len(df_hist.sample_id.unique()), len(df_spec.sample_id.unique())

(793, 775)

# old_id will not work

In [5]:
# old_id never matches
pd.merge(df_hist, df_spec, on='old_id')

,id_x,fork_length_x,total_length,weight_x,age_type_x,river_age_x,notes_x,old_id,life_stage_id_x,sample_id_x,sex_id_x,status_id_x,id_y,fork_length_y,weight_y,river_age_y,notes_y,sample_id_y,sex_id_y,status_id_y,age_type_y,sweep_id,life_stage_id_y,smart_river_age,smart_river_age_type


In [6]:
df_hist = df_hist.drop('old_id', axis=1)
df_spec = df_spec.drop('old_id', axis=1)

# how to identify distinct fish for matching

In [7]:
# distinct matches for 60% of hist fish
df_hist.groupby(['sample_id', 'fork_length', 'weight', 'sex_id', 'life_stage_id', 'river_age', 'status_id'], dropna=False).count().id.describe(percentiles=[.65, .70, .90])

count   14660.000
mean        1.877
std         2.399
min         1.000
50%         1.000
65%         1.000
70%         2.000
90%         4.000
max        50.000
Name: id, dtype: float64

In [8]:
df_hist.groupby(['sample_id', 'fork_length', 'weight', 'sex_id'], dropna=False).count().id.describe(percentiles=[.65, .70, .90])

count   14522.000
mean        1.895
std         2.413
min         1.000
50%         1.000
65%         1.000
70%         2.000
90%         4.000
max        50.000
Name: id, dtype: float64

In [9]:
df_hist.groupby(['sample_id', 'fork_length', 'weight'], dropna=False).count().id.describe(percentiles=[.65, .70, .90])

count   14433.000
mean        1.907
std         2.419
min         1.000
50%         1.000
65%         1.000
70%         2.000
90%         4.000
max        50.000
Name: id, dtype: float64

In [10]:
groupby = ['sample_id', 'fork_length', 'weight', 'sex_id', 'life_stage_id', 'river_age', 'status_id']
df_spec.groupby(groupby, dropna=False).count().id.describe(percentiles=[.30, .35])

count   8770.000
mean       8.514
std       19.040
min        1.000
30%        1.000
35%        2.000
50%        3.000
max      425.000
Name: id, dtype: float64

In [11]:
groupby = ['sample_id', 'fork_length', 'weight', 'sex_id']
df_spec.groupby(groupby, dropna=False).count().id.describe(percentiles=[.30, .35])

count   8768.000
mean       8.516
std       19.042
min        1.000
30%        1.000
35%        2.000
50%        3.000
max      425.000
Name: id, dtype: float64

In [12]:
groupby = ['sample_id', 'fork_length', 'weight']
df_spec.groupby(groupby, dropna=False).count().id.describe(percentiles=[.30, .35])

count   8767.000
mean       8.517
std       19.043
min        1.000
30%        1.000
35%        2.000
50%        3.000
max      425.000
Name: id, dtype: float64

### percentages of distinct fish
* about 60% of historical fish groupings can be uniquely identified by length, weight, and id
* about 30% of specimen fish groupings can be uniquely identified by length, weight, and id
* sex_id differentiates at least one indistinct example, so just add it
* maybe exact matches don't matter - same attributes, same sample, maybe just matching in order would work

In [13]:
df_hist['matching_id'] = df_hist['sample_id'].astype(str) + df_hist['fork_length'].astype(str) + df_hist['weight'].fillna(0).astype(str) + df_hist['sex_id'].fillna(0).astype(str)
df_hist['matching_id'] = df_hist['matching_id'].str.replace('.', '_', regex=False)
df_hist['distinct'] = ~df_hist.matching_id.isin(df_hist[df_hist.matching_id.duplicated()].matching_id.unique())

In [14]:
df_spec['matching_id'] = df_spec['sample_id'].astype(str) + df_spec['fork_length'].astype(str) + df_spec['weight'].fillna(0).astype(str) + df_spec['sex_id'].fillna(0).astype(str)
df_spec['matching_id'] = df_spec['matching_id'].str.replace('.', '_', regex=False)
df_spec['distinct'] = ~df_spec.matching_id.isin(df_spec[df_spec.matching_id.duplicated()].matching_id.unique())

In [15]:
spec_ids = list(df_spec.matching_id.unique())
hist_ids = list(df_hist.matching_id.unique())

In [16]:
# how many matches are distinct?
matches = [x for x in spec_ids if x in hist_ids]

In [17]:
df_spec[df_spec.matching_id.isin(matches)].distinct.value_counts()

False    32015
True       358
Name: distinct, dtype: int64

In [18]:
df_hist[df_hist.matching_id.isin(matches)].distinct.value_counts()

False    4183
True     1041
Name: distinct, dtype: int64

In [19]:
# indistinct matches - from specimen
df_spec[df_spec.matching_id.isin(matches)].shape[0], df_spec.shape[0]

(32373, 74666)

In [20]:
# indistinct matches - from historical
df_hist[df_hist.matching_id.isin(matches)].shape[0], df_hist.shape[0]

(5224, 27524)

In [21]:
# missing from
len([x for x in spec_ids if x not in hist_ids]), len([x for x in hist_ids if x not in spec_ids])
# hist, spec

(6706, 12460)

In [22]:
# matched
len([x for x in spec_ids if x in hist_ids]), len([x for x in hist_ids if x in spec_ids])

(2062, 2062)

In [23]:
# total number of matching ids
len(hist_ids), len(spec_ids)

(14522, 8768)

In [24]:
# distinct exact matches
distinct_matched_hist = list(df_hist[(df_hist.matching_id.isin(matches)) & df_hist.distinct==True].matching_id)
df_spec[(df_spec.matching_id.isin(distinct_matched_hist)) & (df_spec.distinct==True)].shape[0]

271

# let's build a throwaway html to list all of the potential matches

    -- list of sample id with historical and specimen data
    SELECT trapnet_sample.id
        FROM trapnet_sample
            JOIN trapnet_biologicaldetailing ON trapnet_biologicaldetailing.sample_id = trapnet_sample.id
            JOIN trapnet_specimen ON trapnet_specimen.sample_id = trapnet_sample.id
    GROUP BY trapnet_sample.id

In [25]:
# df_matching_id = pd.read_csv(r'.\csv\SQL_matching_id.csv')
# link_template = '<a href="http://127.0.0.1:8000/en/trapnet/samples/{}/view/">{}</a>'

# with open('all_matches.html', 'w') as f:
#     for i, row in df_matching_id.iterrows():
#         pk = row['id']
#         f.write(link_template.format(pk, pk) + '\n')